In [1]:
import pickle

from akida_models import fetch_file

# Fetch pre-processed data for 32 keywords
fname = fetch_file(
    fname='kws_preprocessed_all_words_except_backward_follow_forward.pkl',
    origin="https://data.brainchip.com/dataset-mirror/kws/kws_preprocessed_all_words_except_backward_follow_forward.pkl",
    cache_subdir='datasets/kws')
with open(fname, 'rb') as f:
    [_, _, x_valid, y_valid, _, _, word_to_index, _] = pickle.load(f)

# Preprocessed dataset parameters
num_classes = len(word_to_index)

print("Wanted words and labels:\n", word_to_index)

Wanted words and labels:
 {'six': 23, 'three': 25, 'seven': 21, 'bed': 1, 'eight': 6, 'yes': 31, 'cat': 3, 'on': 18, 'one': 19, 'stop': 24, 'two': 27, 'house': 11, 'five': 7, 'down': 5, 'four': 8, 'go': 9, 'up': 28, 'learn': 12, 'no': 16, 'bird': 2, 'zero': 32, 'nine': 15, 'visual': 29, 'wow': 30, 'sheila': 22, 'marvin': 14, 'off': 17, 'right': 20, 'left': 13, 'happy': 10, 'dog': 4, 'tree': 26, '_silence_': 0}


In [2]:
from tensorflow.keras.models import load_model

# Retrieve the model file from the BrainChip data server
model_file = fetch_file(fname="ds_cnn_kws.h5",
                        origin="https://data.brainchip.com/models/AkidaV2/ds_cnn/ds_cnn_kws.h5",
                        cache_subdir='models')

# Load the native Keras pre-trained model
model_keras = load_model(model_file)
model_keras.summary()

Model: "ds_cnn_kws"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 49, 10, 1)]       0         
                                                                 
 rescaling (Rescaling)       (None, 49, 10, 1)         0         
                                                                 
 conv_0 (Conv2D)             (None, 25, 5, 64)         1600      
                                                                 
 conv_0/BN (BatchNormalizati  (None, 25, 5, 64)        256       
 on)                                                             
                                                                 
 conv_0/relu (ReLU)          (None, 25, 5, 64)         0         
                                                                 
 dw_separable_1 (DepthwiseCo  (None, 25, 5, 64)        576       
 nv2D)                                                  

In [3]:
import numpy as np

from sklearn.metrics import accuracy_score

# Check Keras Model performance
potentials_keras = model_keras.predict(x_valid)
preds_keras = np.squeeze(np.argmax(potentials_keras, 1))

accuracy = accuracy_score(y_valid, preds_keras)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy) + "%")

308/308 [==============================] - 1s 3ms/step
Accuracy: 93.09%


In [4]:
from quantizeml.models import load_model

# Load the pre-trained quantized model
model_file = fetch_file(
    fname="ds_cnn_kws_i8_w8_a8.h5",
    origin="https://data.brainchip.com/models/AkidaV2/ds_cnn/ds_cnn_kws_i8_w8_a8.h5",
    cache_subdir='models')
model_keras_quantized = load_model(model_file)
model_keras_quantized.summary()

# Check Model performance
potentials_keras_q = model_keras_quantized.predict(x_valid)
preds_keras_q = np.squeeze(np.argmax(potentials_keras_q, 1))

accuracy_q = accuracy_score(y_valid, preds_keras_q)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy_q) + "%")

Model: "ds_cnn_kws"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 49, 10, 1)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 49, 10, 1)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 25, 5, 64)         1664      
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 25, 5, 64)        128       
                                                                 
 dw_separable_1 (QuantizedDe  (None, 25, 5, 64)        704       
 pthwiseConv2D)                                                  
                                                                 
 pw_separable_1 (QuantizedCo  (None, 25, 5, 64)        4

In [5]:
# Define QuantizationParams with specific values just for the sake of understanding the JSON
# configuration that follows.

import quantizeml
import json
from quantizeml.models import quantize, QuantizationParams,dump_config
qparams = QuantizationParams(input_weight_bits=8, weight_bits=4, activation_bits=4,
                             per_tensor_activations=True, buffer_bits=24)

# Quantize the model
quantized_model = quantize(model_keras_quantized, qparams=qparams)
quantized_model.summary()

Model: "ds_cnn_kws"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 49, 10, 1)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 49, 10, 1)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 25, 5, 64)         1664      
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 25, 5, 64)        128       
                                                                 
 dw_separable_1 (QuantizedDe  (None, 25, 5, 64)        704       
 pthwiseConv2D)                                                  
                                                                 
 pw_separable_1 (QuantizedCo  (None, 25, 5, 64)        4

In [6]:
from cnn2snn import convert

# Convert the model
model_akida = convert(model_keras_quantized)
model_akida.summary()

C:\Users\msaad\AppData\Roaming\Python\Python38\site-packages\cnn2snn\quantizeml\blocks.py:160: UserWarning: Conversion stops at layer dense_5 because of a dequantizer. The end of the model is ignored:
___________________________________________________
Layer (type)
act_softmax (Activation)

  warnings.warn("Conversion stops" + stop_layer_msg + " because of a dequantizer. "


                Model Summary                 
______________________________________________
Input shape  Output shape  Sequences  Layers
[49, 10, 1]  [1, 1, 33]    1          11    
______________________________________________

_________________________________________________________________
Layer (type)                       Output shape  Kernel shape  

============ SW/conv_0-dense_5/dequantizer (Software) ===========

conv_0 (InputConv2D)               [25, 5, 64]   (5, 5, 1, 64) 
_________________________________________________________________
dw_separable_1 (DepthwiseConv2D)   [25, 5, 64]   (3, 3, 64, 1) 
_________________________________________________________________
pw_separable_1 (Conv2D)            [25, 5, 64]   (1, 1, 64, 64)
_________________________________________________________________
dw_separable_2 (DepthwiseConv2D)   [25, 5, 64]   (3, 3, 64, 1) 
_________________________________________________________________
pw_separable_2 (Conv2D)            [25, 5, 64]   (

In [7]:
import time
class Timer:
    def __init__(self):
        self.start_time = None
        self.end_time = None

    def start(self):
        self.start_time = time.perf_counter()

    def stop(self):
        self.end_time = time.perf_counter()
        return self.end_time - self.start_time

In [8]:
# Check Akida model performance
timer = Timer()
timer.start()
preds_akida = model_akida.predict_classes(x_valid, num_classes=num_classes)
prediction_time = timer.stop()
print(f"Prediction took {prediction_time:.4f} seconds")

accuracy = accuracy_score(y_valid, preds_akida)
print("Accuracy: " + "{0:.2f}".format(100 * accuracy) + "%")

# For non-regression purposes
#assert accuracy > 0.9

Prediction took 2.4964 seconds
Accuracy: 92.87%
